<a href="https://colab.research.google.com/github/AbdulsemedShalo/DDOS-Detection-and-Mitigation/blob/main/DDOSMitigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import random

import time
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

# Load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/AbdulsemedShalo/DDOS-Detection-and-Mitigation/main/Dataset.csv")


data = data.drop("Source.IP", axis=1)
data = data.drop("Destination.IP", axis=1)

label_encoder = preprocessing.LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

# Extraxt the features you want to base the foundation of our model training
features = ['Max_Packet_Length','Fwd_Packet_Length_Max','Flow_Packets_Sec','Flow_Bytes_Sec','Packet_Length_Std'
           ,'Packet_Length_Variance','Flow_IAT_Max','Fwd_IAT_Max','Subflow_Fwd_Bytes','Fwd_Packet_Length_Std'
           ,'Bwd_Packets_Sec','min_seg_size_forward','Init_Win_bytes_backward','Average_Packet_Size'
           ,'Packet_Length_Mean','Fwd_IAT_Total','Flow_IAT_Std','Fwd_IAT_Std','Avg_Fwd_Segment_Size'
           ,'Fwd_Packet_Length_Mean','Fwd_Header_Length','Fwd_IAT_Mean','Flow_IAT_Mean','Idle_Max'
           ,'Idle_Mean','Fwd_Packets_Sec']

# the packet of request to be sent for demonstration
new_traffic = [353,353,0.207573889,9.578052299,114.1646586,13033.56926,
               79000000,79000000,969,119.0835509,0.009884471,0,29200
               ,46.14285714,44.04545455,101000000,18000000,18400000
               ,48.45,48.45,728,5323696.105,5058439.7,79000000,20500000,0.197689418]

# Replace missing values with the mean of the column

data['Idle_Mean'].fillna(data['Idle_Mean'].median(), inplace=True)
data['Idle_Max'].fillna(data['Idle_Max'].median(), inplace=True)

data['Flow_IAT_Mean'].fillna(data['Flow_IAT_Mean'].median(), inplace=True)
data['Flow_IAT_Max'].fillna(data['Flow_IAT_Max'].median(), inplace=True)
data['Flow_IAT_Std'].fillna(data['Flow_IAT_Std'].median(), inplace=True)

data['Fwd_IAT_Total'].fillna(data['Fwd_IAT_Total'].median(), inplace=True)
data['Fwd_IAT_Mean'].fillna(data['Fwd_IAT_Mean'].median(), inplace=True)
data['Fwd_IAT_Std'].fillna(data['Fwd_IAT_Std'].median(), inplace=True)
data['Fwd_IAT_Max'].fillna(data['Fwd_IAT_Max'].median(), inplace=True)

data['Fwd_Packet_Length_Std'].fillna(data['Fwd_Packet_Length_Std'].median(), inplace=True)
data['Fwd_Packet_Length_Max'].fillna(data['Fwd_Packet_Length_Max'].median(), inplace=True)
data['Fwd_Packet_Length_Mean'].fillna(data['Fwd_Packet_Length_Mean'].median(), inplace=True)
data['Fwd_Packets_Sec'].fillna(data['Fwd_Packets_Sec'].median(), inplace=True)


data['Packet_Length_Variance'].fillna(data['Packet_Length_Variance'].median(), inplace=True)
data['Packet_Length_Std'].fillna(data['Packet_Length_Std'].median(), inplace=True)
data['Packet_Length_Mean'].fillna(data['Packet_Length_Mean'].median(), inplace=True)

data['Flow_Bytes_Sec'].fillna(data['Flow_Bytes_Sec'].median(), inplace=True)
data['Flow_Packets_Sec'].fillna(data['Flow_Packets_Sec'].median(), inplace=True)

data['Subflow_Fwd_Bytes'].fillna(data['Subflow_Fwd_Bytes'].median(), inplace=True)
data['Fwd_Header_Length'].fillna(data['Fwd_Header_Length'].median(), inplace=True)
data['Avg_Fwd_Segment_Size'].fillna(data['Avg_Fwd_Segment_Size'].median(), inplace=True)
data['Max_Packet_Length'].fillna(data['Max_Packet_Length'].median(), inplace=True)

data['Average_Packet_Size'].fillna(data['Average_Packet_Size'].median(), inplace=True)
data['Init_Win_bytes_backward'].fillna(data['Init_Win_bytes_backward'].median(), inplace=True)
data['min_seg_size_forward'].fillna(data['min_seg_size_forward'].median(), inplace=True)
data['Bwd_Packets_Sec'].fillna(data['Bwd_Packets_Sec'].median(), inplace=True)

def detectMitigate(requestTraffic):
    # Split the dataset into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(data, data["Label"], test_size=0.2, random_state=1)


    clf = GaussianNB()
    clf.fit(X_train.values, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Model Accuracy:", accuracy)

    prediction = clf.predict(requestTraffic)

    if prediction == 0:
      print("The traffic is not a DDoS attack.")
    else:
      # Create a list of blocked IP addresses
      blocked_ips = []

      if requestTraffic not in blocked_ips:
        # Add the DDOS traffic request to the blocked list
        blocked_ips.append(requestTraffic)
        print("a DDoS attack with accuracy percentage of : ",accuracy)

        with open("blocked_ips.txt", "w") as f:
          for requestTraffic in blocked_ips:
            f.write(str(requestTraffic )+ "\n")
      else:
         print("This request is already blocked.")



def sendRequest():
  numbers = []
  for i in range(27):
    number = random.randint(1, 100)
    if random.random() > 0.5:
      number = float(number)
    numbers.append(number)
  return numbers


# generate and send traffic requests until it is stopped
traffic = np.array(sendRequest())
new_traffic = np.array(new_traffic)




while True:
  print("Request Packet : " , sendRequest())
  detectMitigate(traffic.reshape(1,-1))

  character = input()
  if character == "x":
    print("Packet Sending stopped:")
    time.sleep(1)
    break



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Request Packet :  [33.0, 72, 45.0, 9.0, 82, 5, 29, 81, 8.0, 86, 44.0, 30, 17, 93.0, 59, 6, 47.0, 38.0, 43.0, 26, 19, 12, 70.0, 55.0, 87.0, 76.0, 78.0]
Model Accuracy: 0.9243333333333333
a DDoS attack with accuracy percentage of :  0.9243333333333333



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Request Packet :  [18.0, 37.0, 61, 8.0, 9, 82.0, 27.0, 82.0, 25, 94.0, 68, 9.0, 2, 88.0, 98.0, 40, 62, 25.0, 36, 49, 36.0, 13.0, 19.0, 95.0, 21.0, 47, 22.0]
Model Accuracy: 0.9243333333333333
a DDoS attack with accuracy percentage of :  0.9243333333333333



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Request Packet :  [23, 98.0, 1, 55, 11, 73.0, 93.0, 27.0, 61.0, 11, 93, 71.0, 57.0, 74, 91.0, 44, 53.0, 56.0, 87, 4, 13.0, 85.0, 6.0, 40.0, 22, 90, 6]
Model Accuracy: 0.9243333333333333
a DDoS attack with accuracy percentage of :  0.9243333333333333



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Request Packet :  [88, 3, 93.0, 80.0, 35.0, 79, 73, 51, 1.0, 24, 89.0, 65.0, 90, 50.0, 41, 34.0, 80.0, 61.0, 45.0, 89, 99, 82, 84, 77, 21, 2, 23]
Model Accuracy: 0.9243333333333333
a DDoS attack with accuracy percentage of :  0.9243333333333333
x
Packet Sending stopped:
